<a href="https://colab.research.google.com/github/Cralsic123/DeeplabV3net-comparison/blob/main/DeeplabV3net_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
 46% 7.00M/15.1M [00:00<00:00, 72.5MB/s]
100% 15.1M/15.1M [00:00<00:00, 113MB/s] 


In [3]:
import zipfile
with zipfile.ZipFile('/content/brain-mri-images-for-brain-tumor-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [4]:
import tensorflow as tf
import numpy as np
import cv2



In [5]:
interpreter = tf.lite.Interpreter(model_path="deeplabv3_plus_mobilenet.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [6]:
input_details

[{'name': 'image',
  'index': 0,
  'shape': array([  1, 513, 513,   3], dtype=int32),
  'shape_signature': array([  1, 513, 513,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
def preprocess_image(image_path, target_size):

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)

    return image


In [8]:
input_shape = input_details[0]['shape']
target_size = (input_shape[1], input_shape[2])


## Running Conclusions

In [9]:
def run_inference(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

In [10]:
def postprocess_output(output_data, original_image_shape):
    # Removing dimension and resizing to original image shape
    output_image = np.squeeze(output_data)
    output_image = cv2.resize(output_image, (original_image_shape[1], original_image_shape[0]))

    # Apply threshold to get binary mask
    output_mask = output_image > 0.5

    return output_mask

In [11]:
mri_image_path = "/content/yes/Y1.jpg"
original_image = cv2.imread(mri_image_path)


In [12]:
preprocessed_image = preprocess_image(mri_image_path, target_size)
output_data = run_inference(preprocessed_image)
segmentation_mask = postprocess_output(output_data, original_image.shape)


In [13]:
cv2.imshow("Segmentation Mask", segmentation_mask.astype(np.uint8) * 255)
cv2.waitKey(0)
cv2.destroyAllWindows()

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
